<a href="https://colab.research.google.com/github/JeanCCA/Proyecto-parcial/blob/main/AgenciaDeViajes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-telegram-bot
!apt update
!apt install chromium-chromedriver
!pip install selenium

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubun

In [ ]:
import logging
import os
import sys
import pandas as pd
import json

from telegram import Update, InlineKeyboardMarkup, InlineKeyboardButton
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
from time import sleep

logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)


# SCRAPING DE VUELOS SOLO IDA--------------------------------------------------------------------------------------------------------------||
def Scraping_ida():
  global origen_code
  global destino_code
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')

  driver = webdriver.Chrome(options=options)

  driver.get('https://www.despegar.com.ec/shop/flights/results/oneway/'+origen_code+'/'+destino_code+'/2021-03-20/1/0/0/NA/NA/NA/NA?from=SB&di=1-0&reSearch=true')
  detalles = WebDriverWait(driver, 10)\
      .until(EC.element_to_be_clickable((By.XPATH,
                                        '//*[@id="clusters"]/span[1]/div/span/cluster/div/div')))

  texto = detalles.find_element_by_xpath('//*[@id="clusters"]/span[1]/div/span/cluster/div/div')
  texto = texto.text.split('\n')
  control_precio = 'Precio por adulto' in texto
  origen = detalles.find_element_by_xpath('//*[@id="clusters"]/span[1]/div/span/cluster/div/div/div[1]/div/span/div/div[2]/span/route-choice/div/span[2]/span[1]/route-info-item[2]/span/span/span')
  destino = detalles.find_element_by_xpath('//*[@id="clusters"]/span[1]/div/span/cluster/div/div/div[1]/div/span/div/div[2]/span/route-choice/div/span[2]/span[2]/route-info-item[2]/span/span/span')
  fecha_ida = detalles.find_element_by_xpath('//*[@id="clusters"]/span[1]/div/span/cluster/div/div/div[1]/div/span/div/div[2]/span/route-choice/div/span[1]/span[2]/route-info-item/span/span/span')
  hora_ida = detalles.find_element_by_xpath('//*[@id="clusters"]/span[1]/div/span/cluster/div/div/div[1]/div/span/div/div[2]/span/route-choice/ul/li/route/itinerary/div/div/div[2]/itinerary-element[1]/span/span/span')
  
  if control_precio == False:
    precio = detalles.find_element_by_xpath('//*[@id="clusters"]/span[1]/div/span/cluster/div/div/div[2]/fare/span/span/main-fare/span/span[2]/span/flights-price/span/flights-price-element/span/span/em/span[2]')
  else:
    precio = detalles.find_element_by_xpath('//*[@id="clusters"]/span[1]/div/span/cluster/div/div/div[2]/fare/span/span/fare-details-items/div/item-fare/p/span/flights-price/span/flights-price-element/span/span/em/span[2]')

  origen = origen.text
  destino = destino.text
  fecha_ida = fecha_ida.text
  hora_ida = hora_ida.text
  precio = precio.text

  driver.quit()

  lista = [origen, destino, fecha_ida, hora_ida, precio]

  return lista


# SCRAPING DE VUELOS IDA - VUELTA--------------------------------------------------------------------------------------------------------------||
def Scraping():
  global origen_code
  global destino_code
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')

  driver = webdriver.Chrome(options=options)

  driver.get('https://www.despegar.com.ec/shop/flights/results/roundtrip/'+origen_code+'/'+destino_code+'/2021-03-24/2021-03-31/1/0/0/NA/NA/NA/NA/NA?from=SB&di=1-0&reSearch=true')
  detalles = WebDriverWait(driver, 10)\
      .until(EC.element_to_be_clickable((By.XPATH,
                                        '//*[@id="clusters"]/span[1]/div/span/cluster/div/div')))

  texto = detalles.find_element_by_xpath('//*[@id="clusters"]/span[1]/div/span/cluster/div/div')
  texto = texto.text.split('\n')
  control_precio = 'Precio por adulto' in texto
  origen = detalles.find_element_by_xpath('//*[@id="clusters"]/span[1]/div/span/cluster/div/div/div[1]/div/span/div/div[2]/span[1]/route-choice/div/span[2]/span[1]/route-info-item[2]/span/span/span')
  destino = detalles.find_element_by_xpath('//*[@id="clusters"]/span[1]/div/span/cluster/div/div/div[1]/div/span/div/div[2]/span[1]/route-choice/div/span[2]/span[2]/route-info-item[2]/span/span/span')
  fecha_ida = detalles.find_element_by_xpath('//*[@id="clusters"]/span[1]/div/span/cluster/div/div/div[1]/div/span/div/div[2]/span[1]/route-choice/div/span[1]/span[2]/route-info-item/span/span/span')
  hora_ida = detalles.find_element_by_xpath('/html/body/div[12]/div/div/div/div[2]/div/div[2]/div/div[5]/app-root/app-common/items/div/span[1]/span/cluster/div/div/div[1]/div/span/div/div[2]/span[1]/route-choice/ul/li/route/itinerary/div/div/div[2]/itinerary-element[1]/span/span/span')
  fecha_regreso = detalles.find_element_by_xpath('/html/body/div[12]/div/div/div/div[2]/div/div[2]/div/div[5]/app-root/app-common/items/div/span[1]/span/cluster/div/div/div[1]/div/span/div/div[2]/span[2]/route-choice/div/span[1]/span[2]/route-info-item/span/span/span')
  hora_regreso = detalles.find_element_by_xpath('/html/body/div[12]/div/div/div/div[2]/div/div[2]/div/div[5]/app-root/app-common/items/div/span[1]/span/cluster/div/div/div[1]/div/span/div/div[2]/span[2]/route-choice/ul/li/route/itinerary/div/div/div[2]/itinerary-element[1]/span/span/span')
  
  if control_precio == False:
    precio = detalles.find_element_by_xpath('/html/body/div[12]/div/div/div/div[2]/div/div[2]/div/div[5]/app-root/app-common/items/div/span[1]/span/cluster/div/div/div[2]/fare/span/span/main-fare/span/span[2]/span/flights-price/span/flights-price-element/span/span/em/span[2]')
  else:
    precio = detalles.find_element_by_xpath('/html/body/div[12]/div/div/div/div[2]/div/div[2]/div/div[5]/app-root/app-common/items/div/span[1]/span/cluster/div/div/div[2]/fare/span/span/fare-details-items/div/item-fare/p/span/flights-price/span/flights-price-element/span/span/em/span[2]')

  origen = origen.text
  destino = destino.text
  fecha_ida = fecha_ida.text
  hora_ida = hora_ida.text
  fecha_regreso = fecha_regreso.text
  hora_regreso = hora_regreso.text
  precio = precio.text

  driver.quit()

  lista = [origen, destino, fecha_ida, hora_ida, fecha_regreso, hora_regreso, precio]

  return lista



# FUNCIÓN BUSCAR AEROPUERTO-------------------------------------------------------------------||
def Buscar_Aeropuerto(cod):
  with open('Vuelos.json', encoding='utf8') as contenido:
    ciudades = json.load(contenido)
    aero =''
    for iata in ciudades:
      for i in ciudades[iata]:
        if i['IATA'] == cod:
          aero = i['Aeropuerto']
  return aero

# FUNCIÓN BUSCAR CIUDADES--------------------------------------------------------------------||
def Buscar_Ciudades(cod_pais):
  with open('Vuelos.json', encoding='utf8') as contenido:
    ciudades = json.load(contenido)
    x=0
    aux = ''
    ciu = ''
    for ciudad in ciudades:
      for i in ciudades[ciudad]:
        if i['COD'] == cod_pais:
          x+=1
          aux = ('\t\t\t'+str(x)+'\t\t\t\t\t\t'+ciudad+'\n')
          ciu = ciu + aux
  return ciu

# MÉTODO START------------------------------------------------------------------------------||
def start(update, context):
  boton1 = InlineKeyboardButton(
      text ='Bienvenido a la Agencia de Viajes',
      callback_data='Bienvenido a la Agencia de Viajes'
  )
  nombre = update.effective_user["first_name"]
  update.message.reply_text(
      text = f'{nombre} ha inicializado la Agencia de Viajes \U00002708',
      reply_markup=InlineKeyboardMarkup([
                                         [boton1]
                                         ])
  )
  update.message.reply_text(
      parse_mode="HTML",
      text=f'Hola {nombre} escribe /info para más información.')

# MÉTODO HELP-------------------------------------------------------------------------------||
def help(update, context):
  nombre = update.effective_user["first_name"]
  update.message.reply_text(
      parse_mode="HTML",
      text=f'Hola {nombre}, escribe /info para más información.')

# ||------------------------------------------------------------------MÉTODO RESERVA-------------------------------------------------------------------||
def reserva(update, context):
  global origen_seleccionado
  global destino_seleccionado
  global origen_code
  global destino_code
  global tipo_vuelo
  global vuelo
  global reserva_realizada
  global fecha_vuelo
  global precio_vuelo

  mensaje=''
  aux=''

  with open('Vuelos.json', encoding='utf8') as contenido:
      ciudades = json.load(contenido)

  lista1=['Origen:', 'Destino:', 'Fecha de ida:', 'Hora de ida:', 'Fecha de regreso:', 'Hora de regreso:', 'Duración:', 'Precio Unitario:', 'Precio Total:']
  lista2=['Origen:', 'Destino:', 'Fecha:', 'Hora:', 'Precio Unitario:', 'Precio Total:']
  nombre = update.effective_user["first_name"]

  texto = update.message.text.upper()
  texto = texto.replace('/','')
  control_completa = texto.find('RESERVA COMPLETA ')
  control_ida = texto.find('RESERVA IDA ')
  texto = texto.replace('RESERVA ','')

  if reserva_realizada == 1:
    update.message.reply_text(
        parse_mode="HTML",
        text=f'<b>RESERVA EN PROGRESO</b>\n'+
        f'Para consultar primero confirme o cancele la reserva que está en progreso')

  elif texto == 'RESERVA':
    update.message.reply_text(
        parse_mode="HTML",
        text =f'<b>REALIZAR SU RESERVA</b>\n'+
        f'/reserva <b>IDA</b>: Reserva solo ida\n'+
        f'/reserva <b>COMPLETA</b>: Reserva de ida y vuelta')

# RESERVA DE VUELOS IDA - VUELTA
  elif texto == 'COMPLETA':
    if origen_seleccionado == 0 and destino_seleccionado == 0:
      update.message.reply_text(
        parse_mode="HTML",
        text =f'Por favor, primero seleccione su aeropuerto de origen y destino.')
    
    elif origen_seleccionado == 0 and destino_seleccionado != 0:
       update.message.reply_text(
        parse_mode="HTML",
        text =f'Por favor, primero seleccione su aeropuerto de origen.')
    
    elif origen_seleccionado != 0 and destino_seleccionado == 0:
       update.message.reply_text(
        parse_mode="HTML",
        text =f'Por favor, primero seleccione su aeropuerto de destino.')
       
    else:
      update.message.reply_text(
        parse_mode="HTML",
        text =f'/reserva <b>COMPLETA</b> <b>\"#\"</b>: Especificar el número de asientos\n'+
        f'<b>Ejemplo: </b>/reserva COMPLETA 3')
  
  elif control_completa == 0:
    asientos = texto.replace('COMPLETA ','')
    asientos = int(asientos)

    if origen_seleccionado == 0 and destino_seleccionado == 0:
      update.message.reply_text(
        parse_mode="HTML",
        text =f'Por favor, primero seleccione su aeropuerto de origen y destino.')
    
    elif origen_seleccionado == 0 and destino_seleccionado != 0:
       update.message.reply_text(
        parse_mode="HTML",
        text =f'Por favor, primero seleccione su aeropuerto de origen.')
    
    elif origen_seleccionado != 0 and destino_seleccionado == 0:
       update.message.reply_text(
        parse_mode="HTML",
        text =f'Por favor, primero seleccione su aeropuerto de destino.')
       
    elif asientos > 0 and asientos < 100:
      update.message.reply_text(
          parse_mode="HTML",
          text =f'\U000023F3 Por favor, espere unos momentos mientras realizamos la transacción...')
      lista = Scraping()
      lista.insert(6,'8 días')
      precio = lista[7]
      lista[7] = '$'+precio
      precio = precio.replace('.','')
      precio = int(precio)

      cantidad = asientos
      total = cantidad * precio
      lista.append('$'+str(total))

      for i in range (len(lista)):
        aux = ('<b>'+lista1[i]+'</b>\t'+lista[i]+'\n')
        mensaje = mensaje + aux
        
      update.message.reply_text(
        parse_mode="HTML",
        text =f'{mensaje}')
      update.message.reply_text(
        parse_mode="HTML",
        text =f'<b>*****************************************</b>\n'+
        f'/confirmar: Para confirmar la reserva\n'+
        f'/cancelar: Para cancelar la reserva\n'+
        f'<b>*****************************************</b>\n')
      
      tipo_vuelo = 'IDA - VUELTA'
      vuelo = (lista[0]+' - '+lista[1])
      fecha_vuelo = (lista[2]+', '+lista[3]+'h')
      reserva_realizada = 1
      precio_vuelo = lista[8]
    
    elif asientos > 99:
      update.message.reply_text(
        parse_mode="HTML",
        text =f'Excede la cantidad de asientos...\n'+
        '<b>Máximo de asientos:</b> 99')
    
    elif asientos <= 0:
      update.message.reply_text(
        parse_mode="HTML",
        text =f'Por favor ingrese una cantidad válida.')


# RESERVA VUELOS SOLO IDA
  elif texto == 'IDA':
    if origen_seleccionado == 0 and destino_seleccionado == 0:
      update.message.reply_text(
        parse_mode="HTML",
        text =f'Por favor, primero seleccione su aeropuerto de origen y destino.')
    
    elif origen_seleccionado == 0 and destino_seleccionado != 0:
       update.message.reply_text(
        parse_mode="HTML",
        text =f'Por favor, primero seleccione su aeropuerto de origen.')
    
    elif origen_seleccionado != 0 and destino_seleccionado == 0:
       update.message.reply_text(
        parse_mode="HTML",
        text =f'Por favor, primero seleccione su aeropuerto de destino.')
       
    else:
      update.message.reply_text(
        parse_mode="HTML",
        text =f'/reserva <b>IDA</b> <b>\"#\"</b>: Especificar el número de asientos\n'+
        f'<b>Ejemplo: </b>/reserva IDA 3')
  
  elif control_ida == 0:
    asientos = texto.replace('IDA ','')
    asientos = int(asientos)

    if origen_seleccionado == 0 and destino_seleccionado == 0:
      update.message.reply_text(
        parse_mode="HTML",
        text =f'Por favor, primero seleccione su aeropuerto de origen y destino.')
    
    elif origen_seleccionado == 0 and destino_seleccionado != 0:
       update.message.reply_text(
        parse_mode="HTML",
        text =f'Por favor, primero seleccione su aeropuerto de origen.')
    
    elif origen_seleccionado != 0 and destino_seleccionado == 0:
       update.message.reply_text(
        parse_mode="HTML",
        text =f'Por favor, primero seleccione su aeropuerto de destino.')
       
    elif asientos > 0 and asientos < 100:
      update.message.reply_text(
          parse_mode="HTML",
          text =f'\U000023F3 Por favor, espere unos momentos mientras realizamos la transacción...')
      lista = Scraping_ida()
      precio = lista[4]
      lista[4] = '$'+precio
      precio = precio.replace('.','')
      precio = int(precio)

      cantidad = asientos
      total = cantidad * precio
      lista.append('$'+str(total))

      for i in range (len(lista)):
        aux = ('<b>'+lista2[i]+'</b>\t'+lista[i]+'\n')
        mensaje = mensaje + aux
        
      update.message.reply_text(
        parse_mode="HTML",
        text =f'{mensaje}')
      update.message.reply_text(
        parse_mode="HTML",
        text =f'<b>*****************************************</b>\n'+
        f'/confirmar: Para confirmar la reserva\n'+
        f'/cancelar: Para cancelar la reserva\n'+
        f'<b>*****************************************</b>\n')
      
      tipo_vuelo = 'SOLO IDA'
      vuelo = (lista[0]+' - '+lista[1])
      reserva_realizada = 1
      fecha_vuelo = (lista[2]+', '+lista[3]+'h')
      precio_vuelo = lista[5]
    
    elif asientos > 99:
      update.message.reply_text(
        parse_mode="HTML",
        text =f'Excede la cantidad de asientos...\n'+
        '<b>Máximo de asientos:</b> 99')
    
    elif asientos <= 0:
      update.message.reply_text(
        parse_mode="HTML",
        text =f'Por favor ingrese una cantidad válida')
  

# ||-------------------------------------------------------------AQUÍ ACABA EL MÉTODO RESERVA----------------------------------------------------------||


# ||------------------------------------------------------------------MÉTODO BÚSQUEDA-------------------------------------------------------------------||
def busqueda(update, context):
  global origen_seleccionado
  global destino_seleccionado
  global origen_code
  global destino_code
  aux = ''
  mensaje = ''
  with open('Vuelos.json', encoding='utf8') as contenido:
      ciudades = json.load(contenido)
  texto = update.message.text.upper()
  texto = texto.replace('/','')
  control = texto.find('ORIGEN')
  texto = texto.replace('ORIGEN ','')
  texto = texto.replace('DESTINO ','')
  nombre = update.effective_user["first_name"]

  if reserva_realizada == 1:
    update.message.reply_text(
        parse_mode="HTML",
        text=f'<b>RESERVA EN PROGRESO</b>\n'+
        f'Para continuar primero confirme o cancele la reserva que está en progreso')

  elif texto == 'ORIGEN':
    update.message.reply_text(
        parse_mode="HTML",
        text =f'<b>BÚSQUEDA POR ORIGEN</b>\n'+
        f'/origen <b>IATA</b>: Búsqueda por IATA\n'+
        f'/origen <b>AERO</b>: Búsqueda por aeropuerto\n'+
        f'/origen <b>CIU</b>: Búsqueda por ciudad\n'+
        f'/origen <b>PAIS</b>: Búsqueda por país')
    
  elif texto == 'DESTINO':
    update.message.reply_text(
        parse_mode="HTML",
        text =f'<b>BÚSQUEDA POR DESTINO</b>\n'+
        f'/destino <b>IATA</b>: Búsqueda por IATA\n'+
        f'/destino <b>AERO</b>: Búsqueda por aeropuerto\n'+
        f'/destino <b>CIU</b>: Búsqueda por ciudad\n'+
        f'/destino <b>PAIS</b>: Búsqueda por país')
    
# BÚSQUEDA POR IATA----------------------------------------------------------------------------------------------------||    
  elif texto == 'IATA':
    for iata in ciudades:
      for i in ciudades[iata]:
        aux = (i['IATA']+' \t\t\t\t'+i['Aeropuerto']+'\n')
        mensaje = mensaje + aux
    
    if control == 0:
      update.message.reply_text(
            parse_mode="HTML",
            text=f'<b>*****************************************</b>\n'+
            f'<b>SELECCIONAR ORIGEN POR IATA</b>\n'+
            f'<b>*****************************************</b>\n'+
            f'<b>COD\t\t\t\tNOMBRE DE AEROPUERTO</b>\n'+
            f'{mensaje}')
      update.message.reply_text(
            parse_mode="HTML",
            text=f'Para seleccionar el aeropuerto de origen escribe:\n'+
            f'/origen <b>IATA</b> <b>\"COD\"</b>\n'+
            f'<b>Ejemplo:</b> /origen IATA GYE')
    
    else:
      update.message.reply_text(
            parse_mode="HTML",
            text=f'<b>*****************************************</b>\n'+
            f'<b>SELECCIONAR DESTINO POR IATA</b>\n'+
            f'<b>*****************************************</b>\n'+
            f'<b>COD\t\t\t\tNOMBRE DE AEROPUERTO</b>\n'+
            f'{mensaje}')
      update.message.reply_text(
            parse_mode="HTML",
            text=f'Para seleccionar el aeropuerto de destino escribe:\n'+
            f'/destino <b>IATA</b> <b>\"COD\"</b>\n'+
            f'<b>Ejemplo:</b> /destino IATA GYE')

# BÚSQUEDA POR AERO---------------------------------------------------------------------------------------------------||
  elif texto == 'AERO':
    x = 0
    for aero in ciudades:
      for i in ciudades[aero]:
        x+=1
        aux = ('\t\t\t'+str(x)+' \t\t\t\t\t\t'+i['Aeropuerto']+'\n')
        mensaje = mensaje + aux
    
    if control == 0:
      update.message.reply_text(
            parse_mode="HTML",
            text=f'<b>*****************************************</b>\n'+
            f'<b>SELECCIONAR ORIGEN POR AEROPUERTO</b>\n'+
            f'<b>*****************************************</b>\n'+
            f'<b>NUM\t\t\t\tNOMBRE DE AEROPUERTO</b>\n'+
            f'{mensaje}')
      update.message.reply_text(
            parse_mode="HTML",
            text=f'Para seleccionar el aeropuerto de origen escribe:\n'+
            f'/origen <b>AERO</b> <b>\"#\"</b>\n'+
            f'<b>Ejemplo:</b> /origen AERO 1')
    
    else:
      update.message.reply_text(
            parse_mode="HTML",
            text=f'<b>*****************************************</b>\n'+
            f'<b>SELECCIONAR DESTINO POR AEROPUERTO</b>\n'+
            f'<b>*****************************************</b>\n'+
            f'<b>NUM\t\t\t\tNOMBRE DE AEROPUERTO</b>\n'+
            f'{mensaje}')
      update.message.reply_text(
            parse_mode="HTML",
            text=f'Para seleccionar el aeropuerto de destino escribe:\n'+
            f'/destino <b>AERO</b> <b>\"#\"</b>\n'+
            f'<b>Ejemplo:</b> /destino AERO 1')

    
# BÚSQUEDA POR CIUDAD-----------------------------------------------------------------------------------------------||    
  elif texto == 'CIU':
    for ciudad in ciudades:
      for i in ciudades[ciudad]:
        aux = (i['IATA']+' \t\t\t\t'+ciudad+'\n')
        mensaje = mensaje + aux
    if control == 0:
      update.message.reply_text(
            parse_mode="HTML",
            text=f'<b>*****************************************</b>\n'+
            f'<b>SELECCIONAR ORIGEN POR CIUDAD</b>\n'+
            f'<b>*****************************************</b>\n'+
            f'<b>COD\t\t\t\tCIUDAD</b>\n'+
            f'{mensaje}')
      update.message.reply_text(
            parse_mode="HTML",
            text=f'Para seleccionar el aeropuerto de origen escribe:\n'+
            f'/origen <b>CIU</b> <b>\"ciudad\"</b>\n'+
            f'<b>Ejemplo:</b> /origen CIU guayaquil')
    
    else:
      update.message.reply_text(
            parse_mode="HTML",
            text=f'<b>*****************************************</b>\n'+
            f'<b>SELECCIONAR DESTINO POR CIUDAD</b>\n'+
            f'<b>*****************************************</b>\n'+
            f'<b>COD\t\t\t\tCIUDAD</b>\n'+
            f'{mensaje}')
      update.message.reply_text(
            parse_mode="HTML",
            text=f'Para seleccionar el aeropuerto de destino escribe:\n'+
            f'/destino <b>CIU</b> <b>\"ciudad\"</b>\n'+
            f'<b>Ejemplo:</b> /destino CIU guayaquil')
    
# BÚSQUEDA POR PAÍS------------------------------------------------------------------------------------------------||   
  elif texto == 'PAIS' or texto == 'PAÍS':
    auxiliar = ''
    for pais in ciudades:
      for i in ciudades[pais]:
        if i['País'] == auxiliar:
          auxiliar = i['País']
        else:
          auxiliar = i['País']
          if i['País']== 'Italia' or i['País'] == 'Perú':
            aux = (i['COD']+' \t\t\t\t\t'+i['País']+'\n')
          else:
            aux = (i['COD']+' \t\t\t\t'+i['País']+'\n')
          mensaje = mensaje + aux
    
    if control == 0:
      update.message.reply_text(
            parse_mode="HTML",
            text=f'<b>*****************************************</b>\n'+
            f'<b>SELECCIONAR ORIGEN POR PAÍS</b>\n'+
            f'<b>*****************************************</b>\n'+
            f'<b>COD\t\t\tPAÍS</b>\n'+
            f'{mensaje}')
      update.message.reply_text(
            parse_mode="HTML",
            text=f'Para seleccionar el país de origen escribe:\n'+
            f'/origen <b>PAIS</b> <b>\"COD\"</b>\n'+
            f'<b>Ejemplo:</b> /origen PAIS ECU')
      
    else:
      update.message.reply_text(
            parse_mode="HTML",
            text=f'<b>*****************************************</b>\n'+
            f'<b>SELECCIONAR DESTINO POR PAÍS</b>\n'+
            f'<b>*****************************************</b>\n'+
            f'<b>COD\t\t\tPAÍS</b>\n'+
            f'{mensaje}')
      update.message.reply_text(
            parse_mode="HTML",
            text=f'Para seleccionar el país de destino escribe:\n'+
            f'/destino <b>PAIS</b> <b>\"COD\"</b>\n'+
            f'<b>Ejemplo:</b> /destino PAIS ECU')

# SELECCIONAR AEROPUERTO--------------------------------------------------------------------------------------------||  

# GUAYAQUIL
  elif texto == 'IATA GYE' or texto == 'AERO 1' or texto == 'CIU GUAYAQUIL' or texto == 'PAIS ECU 1' or texto == 'PAÍS ECU 1':
    cod = 'GYE'
    mensaje = Buscar_Aeropuerto(cod)
    if control == 0:
      origen_seleccionado = 1
      origen_code = cod
      if destino_seleccionado == 1:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, ya has seleccionado este aeropuerto como destino.\n'+
            f'Si quieres seleccionarlo como origen, por favor cambia el destino.')
      else:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, has seleccionado como origen al:\n<b>Aeropuerto {mensaje}</b>')
        update.message.reply_text(
            parse_mode="HTML",
            text=f'<b>*****************************************</b>\n'+
            f'/destino: Para seleccionar destino\n'+
            f'/reserva: Para realizar una reserva\n'+
            f'<b>*****************************************</b>')

    else:
      destino_seleccionado = 1
      destino_code = cod
      if origen_seleccionado == 1:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, ya has seleccionado este aeropuerto como origen.\n'+
            f'Si quieres seleccionarlo como destino, por favor cambia el origen.')
      else:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, has seleccionado como destino al:\n<b>Aeropuerto {mensaje}</b>')
        update.message.reply_text(
            parse_mode="HTML",
            text=f'<b>*****************************************</b>\n'+
            f'/origen: Para seleccionar origen\n'+
            f'/reserva: Para realizar una reserva\n'+
            f'<b>*****************************************</b>')



# QUITO
  elif texto == 'IATA UIO'  or texto == 'AERO 2' or texto == 'CIU QUITO' or texto == 'PAIS ECU 2' or texto == 'PAÍS ECU 2':
    cod = 'UIO'
    mensaje = Buscar_Aeropuerto(cod)
    if control == 0:
      origen_seleccionado = 2
      origen_code = cod
      if destino_seleccionado == 2:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, ya has seleccionado este aeropuerto como destino.\n'+
            f'Si quieres seleccionarlo como origen, por favor cambia el destino.')
      else:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, has seleccionado como origen al:\n<b>Aeropuerto {mensaje}</b>')
        update.message.reply_text(
            parse_mode="HTML",
            text=f'<b>*****************************************</b>\n'+
            f'/destino: Para seleccionar destino\n'+
            f'/reserva: Para realizar una reserva\n'+
            f'<b>*****************************************</b>')

    else:
      destino_seleccionado = 2
      destino_code = cod
      if origen_seleccionado == 2:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, ya has seleccionado este aeropuerto como origen.\n'+
            f'Si quieres seleccionarlo como destino, por favor cambia el origen.')
      else:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, has seleccionado como destino al:\n<b>Aeropuerto {mensaje}</b>')
        update.message.reply_text(
            parse_mode="HTML",
            text=f'<b>*****************************************</b>\n'+
            f'/origen: Para seleccionar origen\n'+
            f'/reserva: Para realizar una reserva\n'+
            f'<b>*****************************************</b>')



# BARCELONA
  elif texto == 'IATA BCN' or texto == 'AERO 3' or texto == 'CIU BARCELONA' or texto == 'PAIS ESP 1' or texto == 'PAÍS ESP 1':
    cod = 'BCN'
    mensaje = Buscar_Aeropuerto(cod)
    if control == 0:
      origen_seleccionado = 3
      origen_code = cod
      if destino_seleccionado == 3:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, ya has seleccionado este aeropuerto como destino.\n'+
            f'Si quieres seleccionarlo como origen, por favor cambia el destino.')
      else:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, has seleccionado como origen al:\n<b>Aeropuerto {mensaje}</b>')
        update.message.reply_text(
            parse_mode="HTML",
            text=f'<b>*****************************************</b>\n'+
            f'/destino: Para seleccionar destino\n'+
            f'/reserva: Para realizar una reserva\n'+
            f'<b>*****************************************</b>')

    else:
      destino_seleccionado = 3
      destino_code = cod
      if origen_seleccionado == 3:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, ya has seleccionado este aeropuerto como origen.\n'+
            f'Si quieres seleccionarlo como destino, por favor cambia el origen.')
      else:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, has seleccionado como destino al:\n<b>Aeropuerto {mensaje}</b>')
        update.message.reply_text(
            parse_mode="HTML",
            text=f'<b>*****************************************</b>\n'+
            f'/origen: Para seleccionar origen\n'+
            f'/reserva: Para realizar una reserva\n'+
            f'<b>*****************************************</b>')



# ROMA
  elif texto == 'IATA FCO' or texto == 'AERO 4' or texto == 'CIU ROMA' or texto == 'PAIS ITA 1' or texto == 'PAÍS ITA 1':
    cod = 'FCO'
    mensaje = Buscar_Aeropuerto(cod)
    if control == 0:
      origen_seleccionado = 4
      origen_code = cod
      if destino_seleccionado == 4:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, ya has seleccionado este aeropuerto como destino.\n'+
            f'Si quieres seleccionarlo como origen, por favor cambia el destino.')
      else:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, has seleccionado como origen al:\n<b>Aeropuerto {mensaje}</b>')
        update.message.reply_text(
            parse_mode="HTML",
            text=f'<b>*****************************************</b>\n'+
            f'/destino: Para seleccionar destino\n'+
            f'/reserva: Para realizar una reserva\n'+
            f'<b>*****************************************</b>')

    else:
      destino_seleccionado = 4
      destino_code = cod
      if origen_seleccionado == 4:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, ya has seleccionado este aeropuerto como origen.\n'+
            f'Si quieres seleccionarlo como destino, por favor cambia el origen.')
      else:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, has seleccionado como destino al:\n<b>Aeropuerto {mensaje}</b>')
        update.message.reply_text(
            parse_mode="HTML",
            text=f'<b>*****************************************</b>\n'+
            f'/origen: Para seleccionar origen\n'+
            f'/reserva: Para realizar una reserva\n'+
            f'<b>*****************************************</b>')

# OSLO
  elif texto == 'IATA OSL' or texto == 'AERO 5' or texto == 'CIU OSLO' or texto == 'PAIS NOR 1' or texto == 'PAÍS NOR 1':
    cod = 'OSL'
    mensaje = Buscar_Aeropuerto(cod)
    if control == 0:
      origen_seleccionado = 5
      origen_code = cod
      if destino_seleccionado == 5:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, ya has seleccionado este aeropuerto como destino.\n'+
            f'Si quieres seleccionarlo como origen, por favor cambia el destino.')
      else:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, has seleccionado como origen al:\n<b>Aeropuerto {mensaje}</b>')
        update.message.reply_text(
            parse_mode="HTML",
            text=f'<b>*****************************************</b>\n'+
            f'/destino: Para seleccionar destino\n'+
            f'/reserva: Para realizar una reserva\n'+
            f'<b>*****************************************</b>')

    else:
      destino_seleccionado = 5
      destino_code = cod
      if origen_seleccionado == 5:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, ya has seleccionado este aeropuerto como origen.\n'+
            f'Si quieres seleccionarlo como destino, por favor cambia el origen.')
      else:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, has seleccionado como destino al:\n<b>Aeropuerto {mensaje}</b>')
        update.message.reply_text(
            parse_mode="HTML",
            text=f'<b>*****************************************</b>\n'+
            f'/origen: Para seleccionar origen\n'+
            f'/reserva: Para realizar una reserva\n'+
            f'<b>*****************************************</b>')



# LONDRES
  elif texto == 'IATA LHR' or texto == 'AERO 6' or texto == 'CIU LONDRES' or texto == 'PAIS GBR 1' or texto == 'PAÍS GBR 1':
    cod = 'LHR'
    mensaje = Buscar_Aeropuerto(cod)
    if control == 0:
      origen_seleccionado = 6
      origen_code = cod
      if destino_seleccionado == 6:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, ya has seleccionado este aeropuerto como destino.\n'+
            f'Si quieres seleccionarlo como origen, por favor cambia el destino.')
      else:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, has seleccionado como origen al:\n<b>Aeropuerto {mensaje}</b>')
        update.message.reply_text(
            parse_mode="HTML",
            text=f'<b>*****************************************</b>\n'+
            f'/destino: Para seleccionar destino\n'+
            f'/reserva: Para realizar una reserva\n'+
            f'<b>*****************************************</b>')

    else:
      destino_seleccionado = 6
      destino_code = cod
      if origen_seleccionado == 6:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, ya has seleccionado este aeropuerto como origen.\n'+
            f'Si quieres seleccionarlo como destino, por favor cambia el origen.')
      else:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, has seleccionado como destino al:\n<b>Aeropuerto {mensaje}</b>')
        update.message.reply_text(
            parse_mode="HTML",
            text=f'<b>*****************************************</b>\n'+
            f'/origen: Para seleccionar origen\n'+
            f'/reserva: Para realizar una reserva\n'+
            f'<b>*****************************************</b>')



# BUENOS AIRES
  elif texto == 'IATA EZE' or texto == 'AERO 7' or texto == 'CIU BUENOS AIRES' or texto == 'CIU BUENOS' or texto == 'PAIS ARG 1' or texto == 'PAÍS ARG 1':
    cod = 'EZE'
    mensaje = Buscar_Aeropuerto(cod)
    if control == 0:
      origen_seleccionado = 7
      origen_code = cod
      if destino_seleccionado == 7:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, ya has seleccionado este aeropuerto como destino.\n'+
            f'Si quieres seleccionarlo como origen, por favor cambia el destino.')
      else:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, has seleccionado como origen al:\n<b>Aeropuerto {mensaje}</b>')
        update.message.reply_text(
            parse_mode="HTML",
            text=f'<b>*****************************************</b>\n'+
            f'/destino: Para seleccionar destino\n'+
            f'/reserva: Para realizar una reserva\n'+
            f'<b>*****************************************</b>')

    else:
      destino_seleccionado = 7
      destino_code = cod
      if origen_seleccionado == 7:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, ya has seleccionado este aeropuerto como origen.\n'+
            f'Si quieres seleccionarlo como destino, por favor cambia el origen.')
      else:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, has seleccionado como destino al:\n<b>Aeropuerto {mensaje}</b>')
        update.message.reply_text(
            parse_mode="HTML",
            text=f'<b>*****************************************</b>\n'+
            f'/origen: Para seleccionar origen\n'+
            f'/reserva: Para realizar una reserva\n'+
            f'<b>*****************************************</b>')



# LIMA
  elif texto == 'IATA LIM' or texto == 'AERO 8' or texto == 'CIU LIMA' or texto == 'PAIS PER 1' or texto == 'PAÍS PER 1':
    cod = 'LIM'
    mensaje = Buscar_Aeropuerto(cod)
    if control == 0:
      origen_seleccionado = 8
      origen_code = cod
      if destino_seleccionado == 8:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, ya has seleccionado este aeropuerto como destino.\n'+
            f'Si quieres seleccionarlo como origen, por favor cambia el destino.')
      else:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, has seleccionado como origen al:\n<b>Aeropuerto {mensaje}</b>')
        update.message.reply_text(
            parse_mode="HTML",
            text=f'<b>*****************************************</b>\n'+
            f'/destino: Para seleccionar destino\n'+
            f'/reserva: Para realizar una reserva\n'+
            f'<b>*****************************************</b>')

    else:
      destino_seleccionado = 8
      destino_code = cod
      if origen_seleccionado == 8:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, ya has seleccionado este aeropuerto como origen.\n'+
            f'Si quieres seleccionarlo como destino, por favor cambia el origen.')
      else:
        update.message.reply_text(
            parse_mode="HTML",
            text=f'{nombre}, has seleccionado como destino al:\n<b>Aeropuerto {mensaje}</b>')
        update.message.reply_text(
            parse_mode="HTML",
            text=f'<b>*****************************************</b>\n'+
            f'/origen: Para seleccionar origen\n'+
            f'/reserva: Para realizar una reserva\n'+
            f'<b>*****************************************</b>')



# BÚSQUEDA DE CIUDAD POR PAÍS-----------------------------------------------------------------||
  elif texto == 'PAIS ECU' or texto == 'PAÍS ECU':
    cod_pais = texto.replace('PAIS ','')
    cod_pais = cod_pais.replace('PAÍS ','')
    print(cod_pais)
    mensaje = Buscar_Ciudades(cod_pais)
    update.message.reply_text(
        parse_mode="HTML",
        text=f'<b>*****************************************</b>\n'+
        f'<b>CIUDADES DE ECUADOR</b>\n'+
        f'<b>*****************************************</b>\n'+
        f'<b>NUM\t\t\tCIUDAD</b>\n'+
        f'{mensaje}')
    if control == 0:
      update.message.reply_text(
              parse_mode="HTML",
              text=f'Para seleccionar el aeropuerto de origen escribe:\n'+
              f'/origen <b>PAIS</b> <b>\"COD\"</b> <b>\"#\"</b>\n'+
              f'<b>Ejemplo:</b> /origen PAIS ECU 1')
    else:
      update.message.reply_text(
              parse_mode="HTML",
              text=f'Para seleccionar el aeropuerto de destino escribe:\n'+
              f'/destino <b>PAIS</b> <b>\"COD\"</b> <b>\"#\"</b>\n'+
              f'<b>Ejemplo:</b> /destino PAIS ECU 1')
      
    
  elif texto == 'PAIS ESP' or texto == 'PAÍS ESP':
    cod_pais = texto.replace('PAIS ','')
    cod_pais = cod_pais.replace('PAÍS ','')
    mensaje = Buscar_Ciudades(cod_pais)
    update.message.reply_text(
        parse_mode="HTML",
        text=f'<b>*****************************************</b>\n'+
        f'<b>CIUDADES DE ESPAÑA</b>\n'+
        f'<b>*****************************************</b>\n'+
        f'<b>NUM\t\t\tCIUDAD</b>\n'+
        f'{mensaje}')
    if control == 0:
      update.message.reply_text(
              parse_mode="HTML",
              text=f'Para seleccionar el aeropuerto de origen escribe:\n'+
              f'/origen <b>PAIS</b> <b>\"COD\"</b> <b>\"#\"</b>\n'+
              f'<b>Ejemplo:</b> /origen PAIS ESP 1')
    else:
      update.message.reply_text(
              parse_mode="HTML",
              text=f'Para seleccionar el aeropuerto de destino escribe:\n'+
              f'/destino <b>PAIS</b> <b>\"COD\"</b> <b>\"#\"</b>\n'+
              f'<b>Ejemplo:</b> /destino PAIS ESP 1')
    
  elif texto == 'PAIS ITA' or texto == 'PAÍS ITA':
    cod_pais = texto.replace('PAIS ','')
    cod_pais = cod_pais.replace('PAÍS ','')
    mensaje = Buscar_Ciudades(cod_pais)
    update.message.reply_text(
        parse_mode="HTML",
        text=f'<b>*****************************************</b>\n'+
        f'<b>CIUDADES DE ITALIA</b>\n'+
        f'<b>*****************************************</b>\n'+
        f'<b>NUM\t\t\tCIUDAD</b>\n'+
        f'{mensaje}')
    if control == 0:
      update.message.reply_text(
              parse_mode="HTML",
              text=f'Para seleccionar el aeropuerto de origen escribe:\n'+
              f'/origen <b>PAIS</b> <b>\"COD\"</b> <b>\"#\"</b>\n'+
              f'<b>Ejemplo:</b> /origen PAIS ITA 1')
    else:
      update.message.reply_text(
              parse_mode="HTML",
              text=f'Para seleccionar el aeropuerto de destino escribe:\n'+
              f'/destino <b>PAIS</b> <b>\"COD\"</b> <b>\"#\"</b>\n'+
              f'<b>Ejemplo:</b> /destino PAIS ITA 1')
    
  elif texto == 'PAIS NOR' or texto == 'PAÍS NOR':
    cod_pais = texto.replace('PAIS ','')
    cod_pais = cod_pais.replace('PAÍS ','')
    mensaje = Buscar_Ciudades(cod_pais)
    update.message.reply_text(
        parse_mode="HTML",
        text=f'<b>*****************************************</b>\n'+
        f'<b>CIUDADES DE NORUEGA</b>\n'+
        f'<b>*****************************************</b>\n'+
        f'<b>NUM\t\t\tCIUDAD</b>\n'+
        f'{mensaje}')
    if control == 0:
      update.message.reply_text(
              parse_mode="HTML",
              text=f'Para seleccionar el aeropuerto de origen escribe:\n'+
              f'/origen <b>PAIS</b> <b>\"COD\"</b> <b>\"#\"</b>\n'+
              f'<b>Ejemplo:</b> /origen PAIS NOR 1')
    else:
      update.message.reply_text(
              parse_mode="HTML",
              text=f'Para seleccionar el aeropuerto de destino escribe:\n'+
              f'/destino <b>PAIS</b> <b>\"COD\"</b> <b>\"#\"</b>\n'+
              f'<b>Ejemplo:</b> /destino PAIS NOR 1')
  
  elif texto == 'PAIS GBR' or texto == 'PAÍS GBR':
    cod_pais = texto.replace('PAIS ','')
    cod_pais = cod_pais.replace('PAÍS ','')
    mensaje = Buscar_Ciudades(cod_pais)
    update.message.reply_text(
        parse_mode="HTML",
        text=f'<b>*****************************************</b>\n'+
        f'<b>CIUDADES DE REINO UNIDO</b>\n'+
        f'<b>*****************************************</b>\n'+
        f'<b>NUM\t\t\tCIUDAD</b>\n'+
        f'{mensaje}')
    if control == 0:
      update.message.reply_text(
              parse_mode="HTML",
              text=f'Para seleccionar el aeropuerto de origen escribe:\n'+
              f'/origen <b>PAIS</b> <b>\"COD\"</b> <b>\"#\"</b>\n'+
              f'<b>Ejemplo:</b> /origen PAIS GBR 1')
    else:
      update.message.reply_text(
              parse_mode="HTML",
              text=f'Para seleccionar el aeropuerto de destino escribe:\n'+
              f'/destino <b>PAIS</b> <b>\"COD\"</b> <b>\"#\"</b>\n'+
              f'<b>Ejemplo:</b> /destino PAIS GBR 1')
    
  elif texto == 'PAIS ARG' or texto == 'PAÍS ARG':
    cod_pais = texto.replace('PAIS ','')
    cod_pais = cod_pais.replace('PAÍS ','')
    mensaje = Buscar_Ciudades(cod_pais)
    update.message.reply_text(
        parse_mode="HTML",
        text=f'<b>*****************************************</b>\n'+
        f'<b>CIUDADES DE ARGENTINA</b>\n'+
        f'<b>*****************************************</b>\n'+
        f'<b>NUM\t\t\tCIUDAD</b>\n'+
        f'{mensaje}')
    if control == 0:
      update.message.reply_text(
              parse_mode="HTML",
              text=f'Para seleccionar el aeropuerto de origen escribe:\n'+
              f'/origen <b>PAIS</b> <b>\"COD\"</b> <b>\"#\"</b>\n'+
              f'<b>Ejemplo:</b> /origen PAIS ARG 1')
    else:
      update.message.reply_text(
              parse_mode="HTML",
              text=f'Para seleccionar el aeropuerto de destino escribe:\n'+
              f'/destino <b>PAIS</b> <b>\"COD\"</b> <b>\"#\"</b>\n'+
              f'<b>Ejemplo:</b> /destino PAIS ARG 1')

  elif texto == 'PAIS PER' or texto == 'PAÍS PER':
    cod_pais = texto.replace('PAIS ','')
    cod_pais = cod_pais.replace('PAÍS ','')
    mensaje = Buscar_Ciudades(cod_pais)
    update.message.reply_text(
        parse_mode="HTML",
        text=f'<b>*****************************************</b>\n'+
        f'<b>CIUDADES DE PERÚ</b>\n'+
        f'<b>*****************************************</b>\n'+
        f'<b>NUM\t\t\tCIUDAD</b>\n'+
        f'{mensaje}')
    if control == 0:
      update.message.reply_text(
              parse_mode="HTML",
              text=f'Para seleccionar el aeropuerto de origen escribe:\n'+
              f'/origen <b>PAIS</b> <b>\"COD\"</b> <b>\"#\"</b>\n'+
              f'<b>Ejemplo:</b> /origen PAIS PER 1')
    else:
      update.message.reply_text(
              parse_mode="HTML",
              text=f'Para seleccionar el aeropuerto de destino escribe:\n'+
              f'/destino <b>PAIS</b> <b>\"COD\"</b> <b>\"#\"</b>\n'+
              f'<b>Ejemplo:</b> /destino PAIS PER 1')

# ||-------------------------------------------------------AQUÍ ACABA EL MÉTODO BÚSQUEDA---------------------------------------------------------------||  

# ||---------------------------------------------------------------MÉTODO CONFIRMAR-------------------------------------------------------------------------||
def confirmar(update, context):
  global tipo_vuelo
  global vuelo
  global reserva_realizada
  global origen_seleccionado
  global destino_seleccionado
  global fecha_vuelo
  global precio_vuelo
  global datos_vuelo
  global reserva_confirmada

  nombre = update.effective_user["first_name"]

  if reserva_realizada == 0:
    update.message.reply_text(
        parse_mode="HTML",
        text=f'{nombre}, no has realizado ninguna reserva para confirmar')
    
  elif reserva_realizada == 1:
    update.message.reply_text(
        parse_mode="HTML",
        text=f'<b>RESERVA CONFIRMADA</b>\n'+
        f'<b>Tipo de Vuelo:</b> {tipo_vuelo}\n'+
        f'<b>Vuelo:</b> {vuelo}\n'+
        f'<b>Fecha:</b> {fecha_vuelo}')
    datos_vuelo.append(nombre)
    datos_vuelo.append(vuelo)
    datos_vuelo.append(tipo_vuelo)
    datos_vuelo.append(fecha_vuelo)
    datos_vuelo.append(precio_vuelo)
    reserva_realizada = 0
    origen_seleccionado = 0
    destino_seleccionado = 0
    reserva_confirmada = 1

# ||-------------------------------------------------------AQUÍ ACABA EL MÉTODO CONFIRMAR---------------------------------------------------------------||


# ||---------------------------------------------------------------MÉTODO CONSULTAR-------------------------------------------------------------------------||

def consultar(update, context):
  global tipo_vuelo
  global vuelo
  global reserva_realizada
  global origen_seleccionado
  global destino_seleccionado
  global fecha_vuelo
  global precio_vuelo
  global datos_vuelo

  aux = ''
  mensaje = ''
  nombre = update.effective_user["first_name"]

  if reserva_realizada == 1:
    update.message.reply_text(
        parse_mode="HTML",
        text=f'<b>RESERVA EN PROGRESO</b>\n'+
        f'Para consultar primero confirme o cancele la reserva que está en progreso')

  elif reserva_realizada == 0:

    if reserva_confirmada == 1:
      for i in range(0, len(datos_vuelo), 5):
        aux = ('<b>*****************************************</b>\n\t\t\t\t<b>DATOS DE VUELO</b>\n<b>*****************************************</b>\n<b>Usuario: </b>'+datos_vuelo[i]+'\n<b>Vuelo: </b>'+datos_vuelo[i+1]+'\n<b>Tipo de Vuelo: </b>'+datos_vuelo[i+2]+'\n<b>Fecha: </b>'+datos_vuelo[i+3]+'\n<b>Precio Total: </b>'+datos_vuelo[i+4]+'\n')
        mensaje = mensaje + aux
      
      update.message.reply_text(
        parse_mode="HTML",
        text=f'<b>CONSULTA DE RESERVAS</b>\n'+
        f'{mensaje}')

    
    elif reserva_confirmada == 0:
      update.message.reply_text(
        parse_mode="HTML",
        text=f'\t\t\t\t\t<b>CONSULTA DE RESERVAS</b>\t\t\t\t\t\n'+
        f'{nombre} no has realizado ninguna reserva')


# ||-------------------------------------------------------AQUÍ ACABA EL MÉTODO CONSULTAR---------------------------------------------------------------||



# ||---------------------------------------------------------------MÉTODO CANCELAR-------------------------------------------------------------------------||
def cancelar(update, context):
  global tipo_vuelo
  global vuelo
  global reserva_realizada
  global origen_seleccionado
  global destino_seleccionado

  texto = update.message.text.upper()
  texto = texto.replace('/','')
  texto = texto.replace('CANCELAR ','')
  nombre = update.effective_user["first_name"]

  if reserva_realizada == 0:
    update.message.reply_text(
        parse_mode="HTML",
        text=f'{nombre}, no has realizado ninguna reserva para cancelar')
    
  elif reserva_realizada == 1:

    if texto == 'CANCELAR':
      update.message.reply_text(
        parse_mode="HTML",
        text=f'/cancelar <b>RESERVA</b>: Cancela la reserva, pero mantiene los aeropuertos seleccionados\n'+
        f'/cancelar <b>TODO</b>: Cancela la reserva, y desmarca los aeropuertos seleccionados\n'+
        f'/confirmar: Para confirmar la reserva')
    
    elif texto == 'RESERVA':
      update.message.reply_text(
        parse_mode="HTML",
        text=f'<b>RESERVA CANCELADA</b>\n\n'+
        f'<b>Tipo de Vuelo:</b> {tipo_vuelo}\n'+
        f'<b>Vuelo:</b> {vuelo}\n')
      
      update.message.reply_text(
        parse_mode="HTML",
        text=f'<b>*****************************************</b>\n'+
        f'/start: Para empezar de nuevo\n'+
        f'/info: Para más información\n'+
        f'/reserva: Realizar una reserva diferente\n'+
        f'<b>*****************************************</b>')
      
      reserva_realizada = 0
      
    elif texto == 'TODO':
      update.message.reply_text(
        parse_mode="HTML",
        text=f'<b>RESERVA CANCELADA</b>\n\n'+
        f'<b>Tipo de Vuelo:</b> {tipo_vuelo}\n'+
        f'<b>Vuelo:</b> {vuelo}\n')
      
      update.message.reply_text(
        parse_mode="HTML",
        text=f'<b>*****************************************</b>\n'+
        f'/start: Para empezar de nuevo\n'+
        f'/info: Para más información\n'+
        f'/origen: Buscar aeropuerto de origen\n'+
        f'/destino: Buscar aeropuerto de destino\n'+
        f'<b>*****************************************</b>')
      
      origen_seleccionado = 0
      destino_seleccionado = 0
      reserva_realizada = 0


# ||-------------------------------------------------------AQUÍ ACABA EL MÉTODO CANCELAR---------------------------------------------------------------||


# ||-----------------------------------------------------------------MÉTODO INFO Y HELP-----------------------------------------------------------------------||
def help(update, context):
  nombre = update.effective_user["first_name"]
  update.message.reply_text(
        parse_mode="HTML",
        text=f'<b>*****************************************</b>\n'+
        f'\U0001F50E <b>BÚSQUEDAS</b> \U0001F50D\n'+
        f'<b>*****************************************</b>\n'+
        f'/origen: Buscar aeropuerto de origen.\n'+
        f'/destino: Buscar aeropuerto de destino.\n'+
        f'<b>*****************************************</b>\n'+
        f'\U0001F39F <b>RESERVAS</b> \U0001F39F\n'+
        f'<b>*****************************************</b>\n'+
        f'/reserva: Realiza una reserva.\n'+
        f'/cancelar: Cancela la reserva en progreso.\n'+
        f'/confirmar: Confirma la reserva en progreso.\n'+
        f'/consultar: Consulta las reservas realizadas.\n'+
        f'<b>*****************************************</b>')
  

def info(update, context):
  nombre = update.effective_user["first_name"]
  update.message.reply_text(
        parse_mode="HTML",
        text=f'<b>*****************************************</b>\n'+
        f'\U0001F4CB <b>INFORMACIÓN</b> \U0001F4CB\n'+
        f'<b>*****************************************</b>\n'+
        f'\U00002705 Los comandos funcionan con o sin el \"/\".\n'+
        f'\U00002705 Los comandos pueden estar en mayúsculas o minúsculas.\n'+
        f'\U00002705 Puede colocar las tildes en las palabras, también funciona sin tildes.\n'+
        f'\U00002705 Para realizar una reserva primero debe seleccionar los aeropuertos de origen y destino.\n'+
        f'\U00002705 Una vez presentado los datos de la reserva tiene que confirmar o cancelar la reserva.\n'+
        f'\U00002705 La reserva solo se cancela antes de confirmarla.\n'+
        f'<b>*****************************************</b>\n')
  
  update.message.reply_text(
        parse_mode="HTML",
        text=f'<b>*****************************************</b>\n'+
        f'Escribe /help para listar los comandos disponibles \U0000203C\n'+
        f'<b>*****************************************</b>\n')
# ||-------------------------------------------------------AQUÍ ACABA EL MÉTODO INFO Y HELP---------------------------------------------------------------||

def SinComandos(update, context):
  texto = update.message.text.upper()
  control_origen = texto.find('ORIGEN')
  control_destino = texto.find('DESTINO')
  control_reserva = texto.find('RESERVA')
  control_confirmar = texto.find('CONFIRMAR')
  control_cancelar = texto.find('CANCELAR')
  control_consultar = texto.find('CONSULTAR')
  control_info = texto.find('INFO')
  control_start = texto.find('START')
  control_help = texto.find('HELP')

  if control_origen == 0 or control_destino == 0:
    busqueda(update, context)
  
  elif control_reserva == 0:
    reserva(update, context)
  
  elif control_confirmar == 0:
    confirmar(update, context)

  elif control_cancelar == 0:
    cancelar(update, context)

  elif control_consultar == 0:
    consultar(update, context)
  
  elif control_info == 0:
    info(update, context)

  elif control_start == 0:
    start(update, context)

  elif control_help == 0:
    help(update, context)


# MÉTODO MAIN-------------------------------------------------------------------------------||
def main():
  
  updater = Updater("1591078038:AAHAvrtn7kT0AYCUt_k1KBEz0I-OwIFV1as")

  dispatcher = updater.dispatcher
  dispatcher.add_handler(CommandHandler("start", start))
  dispatcher.add_handler(CommandHandler("help", help))
  dispatcher.add_handler(CommandHandler("origen", busqueda))
  dispatcher.add_handler(CommandHandler("destino", busqueda))
  dispatcher.add_handler(CommandHandler("reserva", reserva))
  dispatcher.add_handler(CommandHandler("cancelar", cancelar))
  dispatcher.add_handler(CommandHandler("confirmar", confirmar))
  dispatcher.add_handler(CommandHandler("consultar", consultar))
  dispatcher.add_handler(CommandHandler("info", info))
  dispatcher.add_handler(MessageHandler(Filters.text, SinComandos))
  dispatcher.add_handler(CommandHandler("help", help))

  updater.start_polling()

  updater.idle()



if __name__ == "__main__":
  origen_seleccionado = 0
  destino_seleccionado = 0
  origen_code = ''
  destino_code = ''
  tipo_vuelo = ''
  vuelo = ''
  fecha_vuelo = ''
  precio_vuelo = ''
  datos_vuelo = []
  reserva_confirmada = 0
  reserva_realizada = 0
  main()


2021-03-16 22:38:42,613 - apscheduler.scheduler - INFO - Scheduler started
